In [1]:
# This section can be uncommented if running on a system without required modules (e.g., UA HPC)
'''import sys
!{sys.executable} -m pip install rasterio'''

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [2]:
img_ref = "05_13_2024_F120_Beans_P4_15m_RGB_dsm"
src_ref = rasterio.open(img_ref + ".tif")
arr_ref = src_ref.read(1)

img_target = "6_24_2024_F120_Bean_P4_15m_RGB_dsm"
src_target = rasterio.open(img_target+".tif")
arr_target = src_target.read(1)

In [ ]:
# Plot the histogram with the specified range
plt.hist(arr_ref.flatten(), bins=20, range=(arr_ref.min(), arr_ref.max()))
plt.title('Histogram of Pixel Values')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

# Plot the histogram with the specified range
plt.hist(arr_ref[arr_ref != -10000].flatten(), bins=20, range=(arr_ref[arr_ref != -10000].min(), arr_ref[arr_ref != -10000].max()))
plt.title('Histogram of Pixel Values')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

# It's clear there's still outliers
# Calculate the quartiles
q1 = np.percentile(arr_ref, 25)
q3 = np.percentile(arr_ref, 75)

# Compute the IQR
iqr = q3 - q1

# Define the outlier range
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter outliers
filtered_arr_ref = arr_ref[(arr_ref >= lower_bound) & (arr_ref <= upper_bound)]

# Plot the histogram with the specified range
plt.hist(filtered_arr_ref[filtered_arr_ref != -10000].flatten(), 
         bins=20, 
         range=(filtered_arr_ref[filtered_arr_ref != -10000].min(), 
                filtered_arr_ref[filtered_arr_ref != -10000].max()))
plt.title('Histogram of Pixel Values')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Let's look at the same thing with the target flight

# Plot the histogram with the specified range
plt.hist(arr_target.flatten(), bins=20, range=(arr_target.min(), arr_target.max()))
plt.title('Histogram of Pixel Values')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

# Plot the histogram with the specified range
plt.hist(arr_target[arr_target != -10000].flatten(), bins=20, range=(arr_target[arr_target != -10000].min(), arr_target[arr_target != -10000].max()))
plt.title('Histogram of Pixel Values')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

# It's clear there's still outliers
# Calculate the quartiles
q1 = np.percentile(arr_target, 25)
q3 = np.percentile(arr_target, 75)

# Compute the IQR
iqr = q3 - q1

# Define the outlier range
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter outliers
filtered_arr_target = arr_target[(arr_target >= lower_bound) & (arr_target <= upper_bound)]

# Plot the histogram with the specified range
plt.hist(filtered_arr_target[filtered_arr_target != -10000].flatten(), 
         bins=20, 
         range=(filtered_arr_target[filtered_arr_target != -10000].min(), 
                filtered_arr_target[filtered_arr_target != -10000].max()))
plt.title('Histogram of Pixel Values')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
'''def filter_outliers_and_save(input_path, output_path, mask_value=-10000):
    with rasterio.open(input_path) as src:
        data = src.read(1)

        # Create a mask for valid data
        valid_data_mask = data != mask_value
        valid_data = data[valid_data_mask]

        # Calculate z-scores for valid data
        z_scores = np.abs(stats.zscore(valid_data))

        # Define a threshold for outliers
        threshold = 3

        # Identify outliers
        outliers = z_scores > threshold

        # Replace outliers with mask_value
        data[valid_data_mask][outliers] = mask_value

        # Write the result to a new file
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=src.height,
            width=src.width,
            count=1,
            dtype=data.dtype,
            crs=src.crs,
            transform=src.transform
        ) as dst:
            dst.write(data, 1)

# Call the function with the input and output file paths
filter_outliers_and_save('05_13_2024_F120_Beans_P4_15m_RGB_dsm.tif', 'bare_soil_filtered.tif')
filter_outliers_and_save('6_24_2024_F120_Bean_P4_15m_RGB_dsm.tif', '6_64_filtered.tif')'''

#these outputs don't correctly handle nodata cells (-10000), but are included in case the code is informative

In [ ]:
# The above filtering doesn't do much to help other than show that outliers are present.
# Let's look at the bare soil image
show(src_ref)

In [ ]:
# Let's look at the array read from the bare soil image
plt.imshow(arr_ref)

In [3]:
# Note that the array has different bounds

# Let's now check the height reported in each DSM at a specified location. 
# To validate this, we can use the known GCP coordinates.
# GCP #14 is at 408957.138, 3659550.914, 361.793

x_position = 408957.138
y_position = 3659550.914
altitude = 361.793

# Get the corresponding pixels
py, px = src_ref.index(x_position, y_position)
window = rasterio.windows.Window(px - 1//2, py - 1//2, 1, 1)
clip = src_ref.read(window=window)
print("The value read at this point is", round(clip[0][0][0],3))
print("The expected value is", altitude)
print("The difference is", round(clip[0][0][0] - altitude,3))

The value read at this point is 361.793
The expected value is 361.793
The difference is -0.0


In [4]:
# Let's now check this point on the DSM that is not bare soil
# Get the corresponding pixels
py, px = src_target.index(x_position, y_position)
window = rasterio.windows.Window(px - 1//2, py - 1//2, 1, 1)
clip = src_target.read(window=window)
print("The value read at this point is", round(clip[0][0][0],3))
print("The expected value is", altitude)
print("The difference is", round(clip[0][0][0] - altitude,3))

The value read at this point is 361.814
The expected value is 361.793
The difference is 0.021


In [5]:
# Here we see that for the same geographic position, the DSM shows a height of 0.021 m above what is expected.
# We want to shift all points that are not -10000 down by this value.
print("The max of the input is", arr_target.max())
print("The min of the input is", arr_target.min())

arr_target_out = arr_target # set values the same initially
arr_target_out = arr_target_out - (clip[0][0][0] - altitude)


print("The max of the output is", arr_target_out.max())
print("The min of the output is", arr_target_out.min())

# Fix the nodata cells
arr_target_out[arr_target_out<-10000] = -10000

print("The max of the output is", arr_target_out.max())
print("The min of the output is", arr_target_out.min())


The max of the input is 362.84256
The min of the input is -10000.0
The max of the output is 362.82138
The min of the output is -10000.021
The max of the output is 362.82138
The min of the output is -10000.0


In [7]:
img_output = img_target + "_adjusted.tif"

with rasterio.open(img_output, 'w', **src_target.profile) as dst:
    dst.write(arr_target_out, indexes = 1)